In [ ]:
pip install faker sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from datetime import datetime
from sqlalchemy.orm import Session, sessionmaker, scoped_session
from faker import Faker
from uuid import uuid4
import random

In [ ]:
!pwd

/content


In [ ]:
fake = Faker('pt_BR')  # criando gerador de dados falsos

In [ ]:
engine = create_engine("sqlite:///foo.db", echo=False)  # conectando ao banco de dados
Base = declarative_base()  # declarando as estruturas, cria as tabelas dentro da classe

class Pessoas(Base):
    __tablename__ = "pessoas"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    email = Column(String)
    birth = Column(DateTime)
    job = Column(String)

class Pessoas2(Base):
    __tablename__ = "pessoas2"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String,index=True)  # index para deixar as consultas mais rapidas
    email = Column(String, index=True, unique=True)  # unique para ter um unico e-mail
    birth = Column(DateTime)
    job = Column(String)

In [ ]:
Pessoas.__table__.drop(engine, checkfirst = True)

In [ ]:
Pessoas2.__table__.drop(engine, checkfirst = True)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)  # criando uma sessao para manipular os dados(exemplo: select, insert e update)
session = Session()

## Comparando tempo de execução da tabela pessoas1(sem o uso do index) da tabela pessoas2(com o uso do index)###  


**1. Inserindo 9999 registros na tabela Pessoas:**

In [ ]:
%%time
# inserindo registros no banco
# cronometrando usando o comando time     
for i in range(9999):  # inserindo 9999 registros
  insert_fake = Pessoas(name = fake.name(), email = f"{uuid4()}{fake.free_email()}", birth = fake.date_of_birth(), job = fake.job())
  session.add(insert_fake)
  session.commit()

CPU times: user 20.4 s, sys: 5.22 s, total: 25.6 s
Wall time: 1min 53s


In [ ]:
result = session.query(Pessoas).count()  # count para confirmar a quantidade de registros
result

9999

In [ ]:
# instanciando registros
julio_user = Pessoas(name="Julio", email="julio01@gmail.com", birth=datetime.fromisoformat('1995-01-15'), job = "vendedor")  # fromisoformat: padrão internacional de data e hora
session.add(julio_user)

In [ ]:
# cronometrando usando o comando timeit, para ver o tempo medio da execucao
%%timeit
session.query(Pessoas).filter_by(name="Julio").all()

1.95 ms ± 474 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**2. Inserindo 9999 registros na tabela Pessoas2:**

In [ ]:
%%time
for i in range(9999):
  insert_fake = Pessoas2(name = fake.name(), email = f"{uuid4()}{fake.free_email()}", birth = fake.date_of_birth(), job = fake.job())
  session.add(insert_fake)
  session.commit()

CPU times: user 20.9 s, sys: 5.98 s, total: 26.9 s
Wall time: 1min 58s


In [ ]:
julio_user2 = Pessoas2(name="Julio", email="julio01@gmail.com", birth=datetime.fromisoformat('1995-01-15'), job = "vendedor")
session.add(julio_user2)

In [ ]:
%%timeit
session.query(Pessoas2).filter_by(name="Julio").all()

435 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**3. Inserindo 5999 registros na tabela Pessoas:**

In [ ]:
%%time
for i in range(5999):
  insert_fake = Pessoas(name = fake.name(), email = f"{uuid4()}{fake.free_email()}", birth = fake.date_of_birth(), job = fake.job())
  session.add(insert_fake)
  session.commit()

CPU times: user 12.6 s, sys: 3.35 s, total: 15.9 s
Wall time: 1min 12s


In [ ]:
julio_user = Pessoas(name="Julio", email="julio01@gmail.com", birth=datetime.fromisoformat('1995-01-15'), job = "vendedor")
session.add(julio_user)

In [ ]:
%%timeit
session.query(Pessoas).filter_by(name="Julio").all()

3.21 ms ± 85.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**4. Inserindo 5999 registros na tabela Pessoas2**

In [ ]:
%%time
for i in range(5999):
  insert_fake = Pessoas2(name = fake.name(), email = f"{uuid4()}{fake.free_email()}", birth = fake.date_of_birth(), job = fake.job())
  session.add(insert_fake)
  session.commit()

CPU times: user 12.5 s, sys: 3.58 s, total: 16.1 s
Wall time: 1min 12s


In [ ]:
julio_user2 = Pessoas2(name="Julio", email="julio01@gmail.com", birth=datetime.fromisoformat('1995-01-15'), job = "vendedor")
session.add(julio_user2)

In [ ]:
%%timeit
session.query(Pessoas2).filter_by(name="Julio").all()

455 µs ± 9.39 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## **Conclusão**



---



* Com a inserção dos 5999 registros, foi possível avaliar o tempo de inserção da  tabela Pessoas (sem o uso de index) e da tabela Pessoas2 (com index)

* Os resultados mostraram que o tempo de inserção na tabela Pessoas aumentou de 1.95 ms para 3.21 ms, enquanto o tempo de inserção na tabela Pessoas2 permaneceu praticamente constante, passando de 435 µs para 455 µs. 

* Esses resultados indicam que, ao indexar uma coluna e fazer pesquisa por ela, o tempo de consulta é praticamente constante, independentemente do tamanho da tabela. Isso ocorre porque o índice permite que o banco de dados localize rapidamente as linhas que correspondem à pesquisa, sem precisar percorrer todas as linhas da tabela.

* Dessa forma, o uso de índices é uma técnica importante para melhorar o desempenho de consultas em grandes tabelas.






**Visualizando registros:**

In [ ]:
for (name,email,birth,job) in session.query(Pessoas.name,Pessoas.email,Pessoas.birth,Pessoas.job).limit(10):
  print(name,email,birth,job)

Sra. Laura Campos 29310411-8895-4a92-988c-5187fabc9746nathannogueira@yahoo.com.br 2018-03-20 00:00:00 Reumatologista
Cauã da Rocha e2c666fc-5016-4d56-841f-c4e130ad91admendeslucas@uol.com.br 1983-06-30 00:00:00 Tecnólogo em processos químicos
Thales Mendes 0fa14804-96b2-4a2f-83a3-c08330217e6eenzomartins@uol.com.br 1916-09-09 00:00:00 Pedagogo
Thiago Peixoto be2a0e40-26bd-4cbf-894f-a547f01cdfdbana-luiza62@bol.com.br 1937-09-14 00:00:00 Pediatra
Nicole Lopes cefbd9e2-725e-4c30-9192-a0953b920a19maria28@hotmail.com 2000-06-09 00:00:00 Capitão
João Felipe Peixoto 502a703c-af1e-4a55-b28f-b00b9c22cb41sophiabarbosa@gmail.com 1956-01-24 00:00:00 Babá
Thiago Caldeira cf0a3077-67b4-4f2f-9b35-417cafa0219fjdias@hotmail.com 1983-07-30 00:00:00 Espeleologista
Clara Silveira f01890bf-d972-4a1d-b258-8ef7ed65b0b4maria-alice31@bol.com.br 2013-11-03 00:00:00 Vistoriador de sinistros
Breno da Cruz 7196c684-9431-4429-b191-acd6f14ab791yasminpereira@bol.com.br 1986-08-17 00:00:00 Coloproctologista
Lavínia More